In [1]:
#Import libraries 
import numpy as np
import math 
from sklearn.svm import SVC
import matplotlib.pyplot as plt 
import time
from sklearn.model_selection import train_test_split
%matplotlib inline



#Split data into training set proper, calibration set and test set
data_train_p, data_cal, target_train_p, target_cal = train_test_split(data_train, target_train, random_state=0)

print("Training Proper Size:", len(data_train_p))
print("Calibration Set Size:",len(data_cal))
print("Test Set Size:",len(data_test))

Training Proper Size: 5468
Calibration Set Size: 1823
Test Set Size: 2007


In [ ]:
#Load USPS Dataset
import h5py
path="usps.h5"
with h5py.File(path, 'r') as usps_data:
    train = usps_data.get('train')
    x_train = train.get('data')[:]
    y_train = train.get('target')[:]
    test = usps_data.get('test')
    x_test = test.get('data')[:]
    y_test = test.get('target')[:]

In [ ]:
x_train

In [ ]:
y_train

In [ ]:
x_test

In [ ]:
y_test

In [ ]:
#Concatenate training and test set
usps_x=np.row_stack((x_train,x_test))
usps_y=np.append(y_train,y_test)
print('usps_x: ' + str(usps_x.shape)) #Total size of the dataset when we concatenate both training and test set
print('usps_y: ' + str(usps_y.shape))

In [ ]:
#Initially complete dataset split into 2,the training set size=80% and the rest are test set
x_train, x_test, y_train, y_test = train_test_split(usps_x, usps_y, random_state=0,train_size=0.80)
print('X_train: ' + str(x_train.shape))
print('Y_train: ' + str(y_train.shape))
print('X_test:  '  + str(x_test.shape))
print('Y_test:  '  + str(y_test.shape))
print("Training  Size:", len(x_train))
print("Test dataset Size:",len(x_test))

In [2]:
def set_variables(x_train,y_train,x_test,y_test,kernel):
    model=SVC(kernel=kernel)
    score=np.zeros(len(x_train)+1)
    p_values=np.zeros((len(x_test),len(set(y_train))))
    pred=np.zeros(len(x_test))
    conf=np.zeros(len(x_test))
    cred=np.zeros(len(x_test))
    sum_p=0
    return SVM, score, p_values, pred, conf, cred, sum_p

In [3]:
def calculate_score(extend_train_x,extend_train_y,model):
    model.fit(extend_train_x,extend_train_y)
    dec=model.decision_function(extend_train_x)
    row=np.arange(0,len(dec))
    result_score=-1*d_func[row,extend_train_y]
    return result_score,model

In [4]:
def svm_tcp(x_train,y_train,x_test,y_test,kernel):
    #Set variables
    model, score, p_values, pred, conf, cred, sum_p = \
    set_variables(x_train,y_train,x_test,y_test,kernel)
    #Loop through test samples
    for i in range(len(x_test)):
        #Create new dataset which is the training set + the test sample
        extend_train_x=np.row_stack((x_train,x_test[i]))
        for j in set(y_train): #Test all possible labels
            #Extend labels with test sample
            extend_train_y=np.append(y_train,j)
            #Calculate conformity scores for each sample in the extended set
            result_score,model=\
                calculate_score(extend_train_x,extend_train_y,model)
            #Calculate p-values of test sample
            p_values[i][j]=np.mean(result_score>=result_score[-1])
        #Use p-values of test sample to calculate various measures
        pred[i]=int(np.argmax(p_values[i]))
        conf[i]=1- p_values[i][np.argsort(p_values[i])[-2]]
        cred[i]=np.max(p_values[i])
        sum_p = sum_p + np.sum(p_values[i]) - p_values[i][y_test[i]]
    false_p_value=sum_p/(2*len(x_test))
    return pred, conf, cred, false_p_value

In [5]:
start_time = time.time()
pred, conf, cred, false_p_value = \
svm_conformal_predictor(data_train,target_train,data_test[0:10],target_test[0:10],"linear" )
print("Total run time is %s seconds ---" % (time.time() - start_time))

--- 168.38916659355164 seconds ---


In [7]:
print("Linear SVM Accuracy: ", np.mean(pred==target_test[0:10]))
print("Linear SVM Avg False p-value: ", false_p_value)
print("Linear Avg Confidence: ",np.mean(conf))
print("Linear Avg Credibility: ",np.mean(cred))

Linear SVM Accuracy:  0.7
Linear SVM Avg False p-value:  0.17447202413603946
Linear Avg Confidence:  0.8196379594075699
Linear Avg Credibility:  0.6471475589687328
